In [1]:
# !pip -q install gradio ibm-watsonx-ai langchain langchain-community langchain-ibm chromadb pypdf pydantic 

In [2]:
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames
from ibm_watsonx_ai import Credentials
from langchain_ibm import WatsonxLLM, WatsonxEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA
import gradio as gr
from pathlib import Path
import os 

# Importing environment variables 
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
IBM_PROJECT_API_KEY = os.getenv("IBM_PROJECT_API_KEY")
IBM_PROJECT_URL = os.getenv("IBM_PROJECT_URL")
IBM_PROJECT_ID = os.getenv("IBM_PROJECT_ID")

### **Task 1**

(This task corresponds with Exercise 1 in the lab “Load Documents Using LangChain for Different Sources” from Module 1)

Capture a screenshot (saved as pdf_loader) that displays both the code used and the first 1000 characters of the content after loading the paper link.

In [4]:
## Document loader
def document_loader(file):
    loader = PyPDFLoader(file)
    loaded_document = loader.load()
    return loaded_document

## Text splitter
def text_splitter(data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=50,
        length_function=len,
    )
    chunks = text_splitter.split_documents(data)
    return chunks

In [5]:
file = Path("/home/vasim/Khatir/IBM-AI-Engineering/13. Project: GenAI Applications with RAG & LangChain/Notes/A_Comprehensive_Review_of_Low_Rank_Adaptation_in_Large_Language_Models_for_Efficient_Parameter_Tuning-1.pdf")

In [6]:
documents = document_loader(file)
print(f"Length of docuemtns: {len(documents)}")
print(f"First 1000 characters in first document:\n")
# print(documents[0].page_content)
print(documents[0].page_content[:1000])

Length of docuemtns: 11
First 1000 characters in first document:

A Comprehensive Review of Low-Rank
Adaptation in Large Language Models for
Efficient Parameter Tuning
September 10, 2024
Abstract
Natural Language Processing (NLP) often involves pre-training large
models on extensive datasets and then adapting them for specific tasks
through fine-tuning. However, as these models grow larger, like GPT-3
with 175 billion parameters, fully fine-tuning them becomes computa-
tionally expensive. We propose a novel method called LoRA (Low-Rank
Adaptation) that significantly reduces the overhead by freezing the orig-
inal model weights and only training small rank decomposition matrices.
This leads to up to 10,000 times fewer trainable parameters and reduces
GPU memory usage by three times. LoRA not only maintains but some-
times surpasses fine-tuning performance on models like RoBERTa, De-
BERTa, GPT-2, and GPT-3. Unlike other methods, LoRA introduces
no extra latency during inference, making 

### **Task 2**

(This task corresponds with Exercise 2 in the lab, “Apply text splitting techniques to enhance model responsiveness.”)

Submit a screenshot (saved as ‘code_splitter.png’) that displays the code used to split the following LATEX code and its corresponding results.

In [7]:
latex_text = r"""
\documentclass{article}

\begin{document}

\maketitle

\section{Introduction}

Large language models (LLMs) are a type of machine learning model that can be trained on vast amounts of text data to generate human-like language. In recent years, LLMs have made significant advances in various natural language processing tasks, including language translation, text generation, and sentiment analysis.

\subsection{History of LLMs}

The earliest LLMs were developed in the 1980s and 1990s, but they were limited by the amount of data that could be processed and the computational power available at the time. In the past decade, however, advances in hardware and software have made it possible to train LLMs on massive datasets, leading to significant improvements in performance.

\subsection{Applications of LLMs}

LLMs have many applications in the industry, including chatbots, content creation, and virtual assistants. They can also be used in academia for research in linguistics, psychology, and computational linguistics.

\end{document}
"""

In [8]:
from langchain.text_splitter import LatexTextSplitter

text_splitter = LatexTextSplitter(
        chunk_size=100,
        chunk_overlap=10,
        length_function=len,
    )

In [9]:
chunks = text_splitter.split_text(latex_text)
chunks

['\\documentclass{article}\n\n\\begin{document}\n\n\\maketitle\n\n\\section{Introduction}\n\nLarge language',
 'language models (LLMs) are a type of machine learning model that can be trained on vast amounts of',
 'of text data to generate human-like language. In recent years, LLMs have made significant advances',
 'advances in various natural language processing tasks, including language translation, text',
 'text generation, and sentiment analysis.\n\n\\subsection{History of LLMs}\n\nThe earliest LLMs were',
 'LLMs were developed in the 1980s and 1990s, but they were limited by the amount of data that could',
 'could be processed and the computational power available at the time. In the past decade, however,',
 'however, advances in hardware and software have made it possible to train LLMs on massive datasets,',
 'datasets, leading to significant improvements in performance.\n\n\\subsection{Applications of',
 'of LLMs}\n\nLLMs have many applications in the industry, including chatbot

### **Task 3**

(This task corresponds with Exercise 1 in the lab “Embed documents using watsonx’s embedding model.”)
Submit a screenshot (saved as ‘embedding.png’) that displays the code used to embed the following sentence and its corresponding results, which display the first five embedding numbers.

query = "How are you?"

In [10]:
## Embedding model
def watsonx_embedding():
    embed_params = {
        EmbedTextParamsMetaNames.TRUNCATE_INPUT_TOKENS: 3,
        EmbedTextParamsMetaNames.RETURN_OPTIONS: {"input_text": True},
    }
    watsonx_embedding = WatsonxEmbeddings(
        model_id="ibm/slate-125m-english-rtrvr",
        url=IBM_PROJECT_URL,
        project_id=IBM_PROJECT_ID,
        apikey=IBM_PROJECT_API_KEY,
        params=embed_params,
    )
    return watsonx_embedding

embedding_model = watsonx_embedding()

In [11]:
query = "How are you?"
embedding_model.embed_query("How are you?")

[-0.06722455,
 -0.023730014,
 0.017487874,
 -0.013195301,
 -0.03958462,
 0.035013206,
 0.026268989,
 -0.016346067,
 0.0017328461,
 0.057110325,
 -0.07210769,
 -0.017297756,
 -0.049611095,
 -0.017423034,
 -0.028412396,
 -0.008028814,
 0.017073661,
 -0.005280184,
 0.025419915,
 -0.033137515,
 0.009124627,
 0.0023437547,
 0.0027022033,
 -0.02301458,
 -0.012872511,
 0.011373145,
 -0.009063827,
 -0.033579566,
 0.0021336055,
 0.039742704,
 -0.007008723,
 0.017540192,
 0.013501204,
 -0.02065632,
 0.03194266,
 0.01087044,
 -0.008687383,
 0.023959257,
 0.0007889831,
 -0.007972035,
 -0.028410012,
 -0.00958747,
 -0.027156787,
 -0.022130527,
 -0.002896206,
 -0.01690262,
 0.077346265,
 0.010192934,
 0.044747256,
 0.016257787,
 0.06652342,
 -0.009298826,
 0.05224959,
 0.0096178325,
 0.047355823,
 -0.038022693,
 0.016534382,
 0.018556956,
 -0.010481163,
 0.0061104945,
 -0.01024595,
 0.051244814,
 -0.05585934,
 -0.008510807,
 0.028069673,
 0.011818047,
 -0.040075816,
 0.029689154,
 -0.08787459,
 0.024

### **Task 4**

(This task corresponds with Exercise 1 in the lab “Create and Configure a Vector Database to Store Document Embeddings”)

Submit a screenshot (saved as ‘vectordb.png’) that displays the code used to create a Chroma vector database that stores the embeddings of the document [new-Policies.txt](13. Project: GenAI Applications with RAG & LangChain/Notes/new-Policies.txt) and then conduct a similarity search for the following query with the top 5 results used.

query = "Smoking policy"

In [12]:
from langchain_community.document_loaders.text import TextLoader

In [15]:
## Document loader
def text_loader(file):
    loader = TextLoader(file.name)
    loaded_document = loader.load()
    return loaded_document

## Text splitter
def text_splitter(data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=50,
        length_function=len,
    )
    chunks = text_splitter.split_documents(data)
    return chunks

## Vector db
def vector_database(chunks):
    embedding_model = watsonx_embedding()
    vectordb = Chroma.from_documents(chunks, embedding_model)
    return vectordb

## Retriever
def create_retriever(file, k=1):
    print("Starting...")
    splits = text_loader(file)
    print("Data is splitted")
    chunks = text_splitter(splits)
    print("Data is chunked")
    vectordb = vector_database(chunks)
    print("Data is add into vectorstore")
    search_kwargs = {"k": k}
    retriever = vectordb.as_retriever(search_kwargs=search_kwargs)
    print("Retriever is created.")
    return retriever

In [16]:
retriever = create_retriever(Path("new-Policies.txt"), k=5)

Starting...
Data is splitted
Data is chunked
Data is add into vectorstore
Retriever is created.


In [17]:
query = "Smoke Policy"
information = retriever.invoke(query)
information

[Document(metadata={'source': 'new-Policies.txt'}, page_content='This policy promotes the safe and responsible use of digital communication tools in line with our values and legal obligations. Employees must understand and comply with this policy. Regular reviews will ensure it remains relevant with changing technology and security standards.\n\n4. Mobile Phone Policy\n\nOur Mobile Phone Policy defines standards for responsible use of mobile devices within the organization to ensure alignment with company values and legal requirements.\n\nAcceptable Use: Mobile devices are primarily for work-related tasks. Limited personal use is allowed if it does not disrupt work responsibilities.\n\nSecurity: Secure your mobile device and credentials. Be cautious with app downloads and links from unknown sources, and report any security issues promptly.\n\nConfidentiality: Avoid sharing sensitive company information via unsecured messaging apps or emails. Exercise caution when discussing company mat

### **Task 5**

(This task corresponds with Exercise 1 in the lab “Develop a Retriever to Fetch Document Segments based on Queries.”)

Submit a screenshot (saved as ‘retriever.png’) that displays the code used to use ChromaDB as a retriever and conduct a similarity search with the top 2 return results. 

The document you can use is [new-Policies.txt](13. Project: GenAI Applications with RAG & LangChain/Notes/new-Policies.txt). 

The query you can use is:

```python
query = "Email policy"
```

In [18]:
retriever = create_retriever(Path("new-Policies.txt"), k=2)

Starting...
Data is splitted
Data is chunked
Data is add into vectorstore
Retriever is created.


In [19]:
query = "Email Policy"
information = retriever.invoke(query)
information

[Document(metadata={'source': 'new-Policies.txt'}, page_content='This policy promotes the safe and responsible use of digital communication tools in line with our values and legal obligations. Employees must understand and comply with this policy. Regular reviews will ensure it remains relevant with changing technology and security standards.\n\n4. Mobile Phone Policy\n\nOur Mobile Phone Policy defines standards for responsible use of mobile devices within the organization to ensure alignment with company values and legal requirements.\n\nAcceptable Use: Mobile devices are primarily for work-related tasks. Limited personal use is allowed if it does not disrupt work responsibilities.\n\nSecurity: Secure your mobile device and credentials. Be cautious with app downloads and links from unknown sources, and report any security issues promptly.\n\nConfidentiality: Avoid sharing sensitive company information via unsecured messaging apps or emails. Exercise caution when discussing company mat

### **Task 6**

(This task corresponds with the lab “Construct a QA Bot That Leverages the LangChain and LLM to Answer Questions from Loaded Documents.”)

Submit a screenshot (saved as ‘QA_bot.png’) that displays the QA bot interface you created based on the lab “Construct a QA Bot That Leverages the LangChain and LLM to Answer Questions from Loaded Documents.” Also, the picture should display that you uploaded a PDF and are asking a query to the bot. 

The PDF you can use is available [here](13. Project: GenAI Applications with RAG & LangChain/Notes/A_Comprehensive_Review_of_Low_Rank_Adaptation_in_Large_Language_Models_for_Efficient_Parameter_Tuning-1.pdf).

The query you can use is:

```python
query = "What this paper is talking about?
```
